In [ ]:
# 这行代码将你的Google云端硬盘挂载到Colab虚拟机上
from google.colab import drive
drive.mount('/content/drive')

# 待办：在你的云端硬盘中输入保存了解压后的作业文件夹的路径，
# 例如 'cs231n/assignments/assignment2/'
FOLDERNAME = 'cs231n/assignments/assignment2/'
assert FOLDERNAME is not None, "[!] 请输入文件夹名称。"

# 现在我们已经挂载了你的云端硬盘，这行代码确保
# Colab虚拟机的Python解释器可以从其中加载
# Python文件
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# 如果CIFAR-10数据集尚未存在，这行代码会将其下载到你的云端硬盘中
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh  # 执行bash脚本下载数据集
%cd /content/drive/My\ Drive/$FOLDERNAME  # 切换回作业主目录

# PyTorch 简介

在本次作业中，你已经编写了大量代码来实现各种神经网络功能。Dropout（随机失活）、Batch Norm（批量归一化）和二维卷积是计算机视觉领域深度学习中的一些核心技术。你也努力让自己的代码高效且向量化。

不过，在本次作业的最后一部分，我们将暂时放下你编写的出色代码库，转而使用两个流行的深度学习框架之一——在本部分中，我们将使用 PyTorch。

## 我们为什么使用深度学习框架？

* 我们的代码现在可以在GPU上运行了！这会让我们的模型训练速度快得多。使用像PyTorch这样的框架时，你可以利用GPU的算力来运行自己定制的神经网络架构，而无需直接编写CUDA代码（这超出了本课程的范围）。
* 在这门课中，我们希望你能为项目做好使用这类框架的准备，这样你就能更高效地进行实验，而不必手动编写每一个想要使用的功能。
* 我们希望你能站在巨人的肩膀上！PyTorch是一个非常出色的框架，它会让你的工作轻松很多，而且现在你已经了解了它们的核心原理，就可以放心地使用它们了:)
* 最后，我们希望你能接触到在学术界或工业界可能会遇到的那种深度学习代码。

## 什么是PyTorch？

PyTorch是一个用于在Tensor（张量）对象上执行动态计算图的系统，这些张量的行为与numpy的ndarray类似。它配备了强大的自动微分引擎，省去了手动反向传播的需求。

## 如何学习PyTorch？

我们以前的一位讲师Justin Johnson制作了一个非常棒的PyTorch[教程](https://github.com/jcjohnson/pytorch-examples)。

你也可以在这里找到详细的[API文档](http://pytorch.org/docs/stable/index.html)。如果有API文档未涵盖的其他问题，[PyTorch论坛](https://discuss.pytorch.org/)是比Stack Overflow更好的提问地方。

# 目录

本次作业包含5个部分。你将在**三个不同的抽象层次**上学习PyTorch，这会帮助你更好地理解它，并为期末项目做好准备。

1. 第一部分，准备工作：我们将使用CIFAR-10数据集。
2. 第二部分，基础PyTorch：**抽象层次1**，我们将直接使用最低级别的PyTorch张量（Tensors）。
3. 第三部分，PyTorch模块API：**抽象层次2**，我们将使用`nn.Module`来定义任意的神经网络架构。
4. 第四部分，PyTorch序列API：**抽象层次3**，我们将使用`nn.Sequential`来非常便捷地定义线性前馈网络。
5. 第五部分，CIFAR-10开放式挑战：请实现你自己的网络，以在CIFAR-10上获得尽可能高的准确率。你可以尝试任何层、优化器、超参数或其他高级功能。

以下是对比表格：

| API（应用程序接口） | 灵活性 | 便捷性 |
|---------------|-------------|-------------|
| 基础（Barebone） | 高 | 低 |
| `nn.Module` | 高 | 中 |
| `nn.Sequential` | 低 | 高 |

# GPU（图形处理器）

你可以在Colab上手动切换到GPU设备，操作步骤是：点击`Runtime（运行时）-> Change runtime type（更改运行时类型）`，然后在`Hardware Accelerator（硬件加速器）`下选择`GPU`。你应该在运行以下导入包的单元格之前完成此操作，因为切换运行时时内核会重启。

In [ ]:
import torch  # 导入PyTorch库
import torch.nn as nn  # 导入PyTorch的神经网络模块
import torch.optim as optim  # 导入PyTorch的优化器模块
from torch.utils.data import DataLoader  # 导入数据加载器，用于批量加载数据
from torch.utils.data import sampler  # 导入采样器，用于数据采样

import torchvision.datasets as dset  # 导入PyTorch的视觉数据集模块
import torchvision.transforms as T  # 导入数据转换模块，用于数据预处理

import numpy as np  # 导入numpy库，用于数值计算

USE_GPU = True  # 标记是否使用GPU
dtype = torch.float32  # 在本教程中我们将全程使用float类型

# 如果使用GPU且CUDA可用，则将设备设置为cuda，否则使用cpu
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# 控制训练损失打印频率的常量
print_every = 100
print('使用的设备：', device)

# 第一部分：准备工作

现在，让我们加载CIFAR-10数据集。首次加载可能需要几分钟时间，但之后文件会被缓存起来。

在本作业的前几部分中，我们需要自己编写代码来下载CIFAR-10数据集、对其进行预处理并以小批量方式迭代；而PyTorch提供了便捷的工具来自动完成这些过程。

In [ ]:
NUM_TRAIN = 49000  # 训练集样本数量

# torchvision.transforms模块提供了数据预处理和数据增强的工具
# 这里我们设置了一个数据转换流程，通过减去RGB通道的均值并除以每个RGB通道的标准差来预处理数据
# 这里我们硬编码了均值和标准差（CIFAR-10数据集的标准预处理参数）
transform = T.Compose([
                T.ToTensor(),  # 将图像转换为PyTorch张量（Tensor）
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # 标准化处理
            ])

# 我们为每个数据拆分（训练集/验证集/测试集）创建一个Dataset对象
# Dataset每次加载一个训练样本，因此我们将每个Dataset包装在DataLoader中
# DataLoader会迭代遍历Dataset并形成小批量数据
# 我们通过向DataLoader传递一个Sampler对象来告诉它如何从底层Dataset中采样，
# 从而将CIFAR-10训练集拆分为训练集和验证集
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)  # 训练集Dataset，应用预处理
loader_train = DataLoader(cifar10_train, batch_size=64,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))  # 训练集数据加载器，采样前49000个样本

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)  # 验证集Dataset（基于训练集的剩余部分）
loader_val = DataLoader(cifar10_val, batch_size=64,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))  # 验证集数据加载器，采样49000到50000之间的样本

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True,
                            transform=transform)  # 测试集Dataset
loader_test = DataLoader(cifar10_test, batch_size=64)  # 测试集数据加载器

# 第二部分：基础 PyTorch

PyTorch 附带了高级 API 来帮助我们方便地定义模型架构，这部分内容我们将在本教程的第二部分介绍。在本节中，我们将从最基础的 PyTorch 元素开始，以更好地理解自动求导引擎。完成这个练习后，你会更能体会到高级模型 API 的便捷之处。

我们将从一个简单的全连接 ReLU 网络开始，该网络有两个隐藏层且不带偏置，用于 CIFAR 分类任务。
这个实现使用 PyTorch 张量（Tensors）上的操作来计算前向传播，并使用 PyTorch 的自动求导功能来计算梯度。你需要理解每一行代码，因为之后你要在此示例基础上编写一个更复杂的版本。

当我们创建一个 `requires_grad=True` 的 PyTorch 张量时，涉及该张量的操作不仅会计算数值，还会在后台构建一个计算图，这使我们能够轻松地通过该图进行反向传播，以计算某些张量相对于下游损失的梯度。具体来说，如果 x 是一个 `x.requires_grad == True` 的张量，那么在反向传播之后，`x.grad` 将是另一个张量，其中存储着 x 相对于最终标量损失的梯度。

### PyTorch张量：Flatten函数
PyTorch张量在概念上类似于numpy数组：它是一个n维数字网格，并且和numpy一样，PyTorch提供了许多函数来高效地对张量进行操作。作为一个简单的例子，我们在下面提供了一个`flatten`函数，该函数对图像数据进行重塑，以便在全连接神经网络中使用。

回想一下，图像数据通常存储在形状为N x C x H x W的张量中，其中：
* N是数据点的数量
* C是通道的数量
* H是中间特征图的高度（以像素为单位）
* W是中间特征图的宽度（以像素为单位）

当我们进行诸如二维卷积之类的操作时，这种数据表示方式是合适的，因为卷积操作需要了解中间特征彼此之间的空间位置关系。然而，当我们使用全连接仿射层处理图像时，我们希望每个数据点由一个单一的向量来表示——将数据的不同通道、行和列分开已经没有意义了。因此，我们使用“flatten”操作将每个表示中的`C x H x W`值折叠成一个单一的长向量。下面的flatten函数首先从给定的一批数据中读取N、C、H和W的值，然后返回该数据的一个“视图（view）”。“view”类似于numpy中的“reshape”方法：它将x的维度重塑为N x ??，其中??可以是任意值（在这种情况下，它将是C x H x W，但我们不需要显式地指定）。

In [ ]:
def flatten(x):
    N = x.shape[0]  # 获取N（数据点数量），x的形状为(N, C, H, W)
    return x.view(N, -1)  # 将每个图像的C*H*W值"展平"为一个单一向量，-1表示自动计算该维度大小

def test_flatten():
    # 创建一个形状为(2, 1, 3, 2)的张量（2个数据点，1个通道，高3，宽2）
    x = torch.arange(12).view(2, 1, 3, 2)
    print('展平前：', x)
    print('展平后：', flatten(x))

test_flatten()  # 调用测试函数

### 基础PyTorch：两层网络

这里我们定义一个函数`two_layer_fc`，它对一批图像数据执行两层全连接ReLU网络的前向传播。定义完前向传播后，我们通过让全零数据流经网络，来检查它是否会崩溃以及是否能生成正确形状的输出。

你不需要在这里编写任何代码，但阅读并理解这个实现很重要。

In [ ]:
import torch.nn.functional as F  # 导入有用的无状态函数（如激活函数等）

def two_layer_fc(x, params):
    """
    一个全连接神经网络；架构为：
    全连接层 -> ReLU激活 -> 全连接层。
    注意：此函数仅定义前向传播；
    PyTorch会为我们处理反向传播。

    网络的输入将是一批数据，形状为
    (N, d1, ..., dM)，其中d1 * ... * dM = D。隐藏层将有H个单元，
    输出层将为C个类别生成分数。

    输入：
    - x：形状为(N, d1, ..., dM)的PyTorch张量，代表一批输入数据。
    - params：网络权重的PyTorch张量列表[w1, w2]；
      w1的形状为(D, H)，w2的形状为(H, C)。

    返回：
    - scores：形状为(N, C)的PyTorch张量，代表输入数据x的分类分数。
    """
    # 首先展平图像
    x = flatten(x)  # 形状：[batch_size, C x H x W]

    w1, w2 = params  # 从参数列表中获取权重

    # 前向传播：使用张量操作计算预测值y。由于w1和
    # w2的requires_grad=True，涉及这些张量的操作会使
    # PyTorch构建计算图，从而实现梯度的自动计算。
    # 因为我们不再手动实现反向传播，所以不需要保留中间值的引用。
    # 也可以使用`.clamp(min=0)`，与F.relu()等价
    x = F.relu(x.mm(w1))  # 第一层全连接 + ReLU激活
    x = x.mm(w2)  # 第二层全连接
    return x


def two_layer_fc_test():
    hidden_layer_size = 42  # 隐藏层大小
    # 创建输入张量：批大小64，特征维度50
    x = torch.zeros((64, 50), dtype=dtype)
    # 初始化权重张量
    w1 = torch.zeros((50, hidden_layer_size), dtype=dtype)
    w2 = torch.zeros((hidden_layer_size, 10), dtype=dtype)
    # 计算网络输出分数
    scores = two_layer_fc(x, [w1, w2])
    print(scores.size())  # 应该输出 [64, 10]

two_layer_fc_test()  # 调用测试函数

### 基础PyTorch：三层卷积网络

在这里，你需要完成函数`three_layer_convnet`的实现，该函数将执行一个三层卷积网络的前向传播。和上面一样，我们可以通过让全零数据流经网络来立即测试实现是否正确。该网络应具有以下架构：

1. 一个带偏置的卷积层，包含`channel_1`个滤波器，每个滤波器的形状为`KW1 x KH1`，零填充为2
2. ReLU非线性激活
3. 一个带偏置的卷积层，包含`channel_2`个滤波器，每个滤波器的形状为`KW2 x KH2`，零填充为1
4. ReLU非线性激活
5. 带偏置的全连接层，生成为C个类别对应的分数

注意：在全连接层之后**没有softmax激活**：这是因为PyTorch的交叉熵损失会为你执行softmax激活，将这一步骤整合在一起可以提高计算效率。

**提示**：关于卷积操作：参考http://pytorch.org/docs/stable/nn.html#torch.nn.functional.conv2d；注意卷积滤波器的形状！

In [ ]:
def three_layer_convnet(x, params):
    """
    执行上述架构定义的三层卷积网络的前向传播。

    输入：
    - x：形状为(N, 3, H, W)的PyTorch张量，表示一批图像（N为批大小，3为RGB通道数）
    - params：包含网络权重和偏置的PyTorch张量列表；应包含以下内容：
      - conv_w1：形状为(channel_1, 3, KH1, KW1)的PyTorch张量，为第一层卷积层的权重
        （channel_1为输出通道数，3为输入通道数，KH1、KW1为滤波器高和宽）
      - conv_b1：形状为(channel_1,)的PyTorch张量，为第一层卷积层的偏置
      - conv_w2：形状为(channel_2, channel_1, KH2, KW2)的PyTorch张量，为第二层卷积层的权重
        （channel_2为该层输出通道数，channel_1为输入通道数）
      - conv_b2：形状为(channel_2,)的PyTorch张量，为第二层卷积层的偏置
      - fc_w：全连接层的权重PyTorch张量。你能弄清楚它的形状应该是什么吗？
        （提示：输入维度为channel_2 * H' * W'，输出维度为C）
      - fc_b：全连接层的偏置PyTorch张量。你能弄清楚它的形状应该是什么吗？
        （提示：形状应为(C,)，C为类别数）

    返回：
    - scores：形状为(N, C)的PyTorch张量，表示x的分类分数
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params  # 从参数列表中解析各层参数
    scores = None  # 初始化分类分数
    ################################################################################
    # 待办：实现三层卷积网络的前向传播。                                             #
    ################################################################################

    ################################################################################
    #                                 你的代码结束                                  #
    ################################################################################
    return scores

在定义好上述卷积网络的前向传播后，运行下面的单元格来测试你的实现。

当你运行这个函数时，输出的分数（scores）应该具有（64，10）的形状。

In [ ]:
def three_layer_convnet_test():
    # 创建输入张量：批大小64，3个通道（RGB），图像尺寸32x32
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)

    # 第一层卷积层参数：6个输出通道，3个输入通道，卷积核大小5x5
    conv_w1 = torch.zeros((6, 3, 5, 5), dtype=dtype)  # [输出通道数, 输入通道数, 卷积核高度, 卷积核宽度]
    conv_b1 = torch.zeros((6,))  # 输出通道数对应的偏置

    # 第二层卷积层参数：9个输出通道，6个输入通道（与上层输出通道数一致），卷积核大小3x3
    conv_w2 = torch.zeros((9, 6, 3, 3), dtype=dtype)  # [输出通道数, 输入通道数, 卷积核高度, 卷积核宽度]
    conv_b2 = torch.zeros((9,))  # 输出通道数对应的偏置

    # 全连接层参数：需要根据两层卷积后的张量形状计算输入维度
    # 此处假设两层卷积后张量形状为[9, 32, 32]（实际需根据卷积参数重新计算）
    fc_w = torch.zeros((9 * 32 * 32, 10))  # 输入维度为卷积输出的展平尺寸，输出维度为10（类别数）
    fc_b = torch.zeros(10)  # 10个类别的偏置

    # 计算三层卷积网络的输出分数
    scores = three_layer_convnet(x, [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b])
    print(scores.size())  # 应该输出 [64, 10]（64个样本，10个类别分数）

# 调用测试函数
three_layer_convnet_test()

### 基础PyTorch：初始化
让我们编写几个工具方法来初始化模型的权重矩阵。

- `random_weight(shape)` 使用Kaiming归一化方法初始化权重张量。
- `zero_weight(shape)` 初始化全零权重张量。适用于实例化偏置参数。

`random_weight`函数使用Kaiming正态初始化方法，相关描述见于：

He et al, *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [ ]:
def random_weight(shape):
    """
    为权重创建随机张量；设置requires_grad=True意味着我们
    希望在反向传播过程中为这些张量计算梯度。
    我们使用Kaiming归一化：sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # 全连接层权重
        fan_in = shape[0]  # 输入特征数
    else:
        # 卷积层权重 [输出通道数, 输入通道数, 核高, 核宽]
        fan_in = np.prod(shape[1:])  # 输入特征数（计算输入通道数与核尺寸的乘积）
    # randn生成标准正态分布的随机数
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True  # 启用梯度计算
    return w

def zero_weight(shape):
    # 创建全零张量，启用梯度计算
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# 创建一个形状为[3 x 5]的权重
# 如果你使用GPU，应该看到类型为`torch.cuda.FloatTensor`
# 否则应该是`torch.FloatTensor`
random_weight((3, 5))

### 基础PyTorch：检查准确率
在训练模型时，我们会使用以下函数来检查模型在训练集或验证集上的准确率。

检查准确率时，我们不需要计算任何梯度；因此，在计算分数时，我们不需要PyTorch为我们构建计算图。为了阻止图的构建，我们将计算放在`torch.no_grad()`上下文管理器的作用域内。

In [ ]:
def check_accuracy_part2(loader, model_fn, params):
    """
    检查分类模型的准确率。

    输入：
    - loader：我们要检查的数据分割对应的DataLoader
    - model_fn：执行模型前向传播的函数，签名为scores = model_fn(x, params)
    - params：给出模型参数的PyTorch张量列表

    返回：无返回值，但会打印模型的准确率
    """
    # 判断是验证集还是测试集（根据数据集的train属性）
    split = 'val' if loader.dataset.train else 'test'
    print('正在检查%s集上的准确率' % split)
    num_correct, num_samples = 0, 0  # 正确预测数和总样本数
    # 禁用梯度计算（检查准确率时不需要）
    with torch.no_grad():
        for x, y in loader:
            # 将输入数据移动到指定设备（如GPU）并转换为指定数据类型
            x = x.to(device=device, dtype=dtype)
            # 将标签移动到指定设备并转换为长整数类型
            y = y.to(device=device, dtype=torch.int64)
            # 计算模型输出分数
            scores = model_fn(x, params)
            # 找到分数最高的类别作为预测结果
            _, preds = scores.max(1)
            # 累加正确预测的数量和总样本数量
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        # 计算准确率
        acc = float(num_correct) / num_samples
        print('正确数：%d / 总数：%d（准确率：%.2f%%）' % (num_correct, num_samples, 100 * acc))

### 基础PyTorch：训练循环
我们现在可以搭建一个基本的训练循环来训练我们的网络。我们将使用不带动量的随机梯度下降来训练模型。我们会使用`torch.functional.cross_entropy`来计算损失；你可以[在这里查看相关说明](http://pytorch.org/docs/stable/nn.html#cross-entropy)。

训练循环接收神经网络函数、初始化参数列表（在我们的示例中是`[w1, w2]`）和学习率作为输入。

In [ ]:
def train_part2(model_fn, params, learning_rate):
    """
    在CIFAR-10数据集上训练模型。

    输入：
    - model_fn：执行模型前向传播的Python函数。
      函数签名应为scores = model_fn(x, params)，其中x是图像数据的PyTorch张量，
      params是给出模型权重的PyTorch张量列表，scores是形状为(N, C)的PyTorch张量，
      给出x中元素的分类分数。
    - params：给出模型权重的PyTorch张量列表
    - learning_rate：用于SGD的学习率（Python标量）

    返回：无
    """
    for t, (x, y) in enumerate(loader_train):
        # 将数据移动到合适的设备（GPU或CPU）
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        # 前向传播：计算分数和损失
        scores = model_fn(x, params)
        loss = F.cross_entropy(scores, y)

        # 反向传播：PyTorch会找出计算图中所有requires_grad=True的张量，
        # 使用反向传播计算损失相对于这些张量的梯度，并将梯度存储在每个张量的.grad属性中。
        loss.backward()

        # 更新参数。我们不希望通过参数更新进行反向传播，
        # 因此将更新操作放在torch.no_grad()上下文管理器中，以防止构建计算图。
        with torch.no_grad():
            for w in params:
                w -= learning_rate * w.grad  # 参数更新公式

                # 在反向传播后手动清零梯度
                w.grad.zero_()

        # 定期打印训练信息和验证集准确率
        if t % print_every == 0:
            print('迭代次数 %d，损失 = %.4f' % (t, loss.item()))
            check_accuracy_part2(loader_val, model_fn, params)
            print()

### 基础PyTorch：训练一个两层网络
现在我们可以运行训练循环了。我们需要为全连接权重`w1`和`w2`显式分配张量。

CIFAR的每个小批量有64个样本，因此张量形状为`[64, 3, 32, 32]`。

展平后，`x`的形状应为`[64, 3 * 32 * 32]`。这将是`w1`第一维度的大小。
`w1`的第二维度是隐藏层大小，它也将是`w2`的第一维度。

最后，网络的输出是一个10维向量，代表10个类别上的概率分布。

你不需要调整任何超参数，但训练一个轮次后，你应该能看到准确率超过40%。

In [ ]:
hidden_layer_size = 4000  # 隐藏层大小
learning_rate = 1e-2  # 学习率

# 初始化第一层权重：输入维度为3*32*32（展平后的图像），输出维度为隐藏层大小
w1 = random_weight((3 * 32 * 32, hidden_layer_size))
# 初始化第二层权重：输入维度为隐藏层大小，输出维度为10（CIFAR-10的类别数）
w2 = random_weight((hidden_layer_size, 10))

# 训练两层全连接网络
train_part2(two_layer_fc, [w1, w2], learning_rate)

### 基础PyTorch：训练一个卷积网络

在下面的内容中，你需要使用上面定义的函数在CIFAR数据集上训练一个三层卷积网络。该网络应具有以下架构：

1. 带偏置的卷积层，包含32个5x5的滤波器，零填充为2
2. ReLU激活函数
3. 带偏置的卷积层，包含16个3x3的滤波器，零填充为1
4. ReLU激活函数
5. 带偏置的全连接层，用于计算10个类别的分数

你应该使用上面定义的`random_weight`函数初始化权重矩阵，使用上面的`zero_weight`函数初始化偏置向量。

你不需要调整任何超参数，但如果一切正常，训练一个轮次后你应该能达到42%以上的准确率。

In [ ]:
learning_rate = 3e-3  # 学习率

channel_1 = 32  # 第一层卷积输出通道数
channel_2 = 16  # 第二层卷积输出通道数

# 初始化各层参数（初始化为None）
conv_w1 = None  # 第一层卷积权重
conv_b1 = None  # 第一层卷积偏置
conv_w2 = None  # 第二层卷积权重
conv_b2 = None  # 第二层卷积偏置
fc_w = None     # 全连接层权重
fc_b = None     # 全连接层偏置

################################################################################
# 待办：初始化三层卷积网络的参数。                                              #
################################################################################

################################################################################
#                                 你的代码结束                                  #
################################################################################

# 参数列表
params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
# 训练三层卷积网络
train_part2(three_layer_convnet, params, learning_rate)

# 第三部分. PyTorch 模块 API

基础的 PyTorch 要求我们手动跟踪所有的参数张量。对于只有几个张量的小型网络来说这没什么问题，但在更大的网络中跟踪数十或数百个张量会极其不便且容易出错。

PyTorch 提供了 `nn.Module` API，让你可以定义任意的网络架构，同时自动为你跟踪所有可学习的参数。在第二部分中，我们自己实现了 SGD。PyTorch 还提供了 `torch.optim` 包，其中实现了所有常见的优化器，如 RMSProp、Adagrad 和 Adam。它甚至支持近似的二阶方法，如 L-BFGS！你可以参考[文档](http://pytorch.org/docs/master/optim.html)了解每个优化器的确切规格。

要使用 Module API，请遵循以下步骤：

1. 继承 `nn.Module`。给你的网络类起一个直观的名字，比如 `TwoLayerFC`。

2. 在构造函数 `__init__()` 中，将所有需要的层定义为类属性。像 `nn.Linear` 和 `nn.Conv2d` 这样的层对象本身就是 `nn.Module` 的子类，并且包含可学习的参数，因此你不必自己实例化原始张量。`nn.Module` 会为你跟踪这些内部参数。参考[文档](http://pytorch.org/docs/master/nn.html)了解数十种内置层的更多信息。**警告**：不要忘记先调用 `super().__init__()`！

3. 在 `forward()` 方法中，定义网络的*连接方式*。你应该将在 `__init__` 中定义的属性用作函数调用，接收张量作为输入并输出“转换后的”张量。不要在 `forward()` 中创建任何带有可学习参数的新层！所有这些层都必须在 `__init__` 中预先声明。

定义好 Module 子类后，你可以将其实例化为一个对象，并像第二部分中的神经网络前向函数一样调用它。

### 模块 API：两层网络
下面是一个两层全连接网络的具体示例：

In [ ]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()  # 调用父类nn.Module的构造函数
        # 将层对象分配为类属性
        self.fc1 = nn.Linear(input_size, hidden_size)  # 第一层全连接层
        # nn.init包包含便捷的初始化方法
        # http://pytorch.org/docs/master/nn.html#torch-nn-init
        nn.init.kaiming_normal_(self.fc1.weight)  # 使用Kaiming正态分布初始化第一层权重
        self.fc2 = nn.Linear(hidden_size, num_classes)  # 第二层全连接层
        nn.init.kaiming_normal_(self.fc2.weight)  # 使用Kaiming正态分布初始化第二层权重

    def forward(self, x):
        # forward方法始终定义网络的连接方式
        x = flatten(x)  # 展平输入张量
        # 前向传播：第一层 -> ReLU激活 -> 第二层
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores

def test_TwoLayerFC():
    input_size = 50  # 输入特征维度
    # 创建输入张量：批大小64，特征维度50
    x = torch.zeros((64, input_size), dtype=dtype)
    # 实例化两层全连接网络：输入维度50，隐藏层大小42，输出类别数10
    model = TwoLayerFC(input_size, 42, 10)
    scores = model(x)  # 计算输出分数
    print(scores.size())  # 应该输出 [64, 10]
test_TwoLayerFC()  # 调用测试函数

### 模块 API：三层卷积网络
现在轮到你实现一个三层卷积网络，其后跟随一个全连接层。网络架构应与第二部分相同：

1. 包含`channel_1`个5x5滤波器的卷积层，零填充为2
2. ReLU激活函数
3. 包含`channel_2`个3x3滤波器的卷积层，零填充为1
4. ReLU激活函数
5. 输出到`num_classes`个类别的全连接层

你应该使用Kaiming正态初始化方法初始化模型的权重矩阵。

**提示**：http://pytorch.org/docs/stable/nn.html#conv2d

实现完三层卷积网络后，`test_ThreeLayerConvNet`函数将运行你的实现；它应打印输出分数的形状为`(64, 10)`。

In [ ]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        ########################################################################
        # 待办：按照上面定义的架构，设置三层卷积网络所需的层。                    #
        ########################################################################

        ########################################################################
        #                          你的代码结束                                 #
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # 待办：实现三层卷积网络的前向传播函数。你应该使用在__init__中定义的层，  #
        # 并在forward()中指定这些层的连接方式                                   #
        ########################################################################

        ########################################################################
        #                             你的代码结束                              #
        ########################################################################
        return scores


def test_ThreeLayerConvNet():
    # 创建输入张量：批大小64，图像尺寸[3, 32, 32]（3通道，32x32像素）
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)
    # 实例化三层卷积网络：输入通道3，第一层输出通道12，第二层输出通道8，类别数10
    model = ThreeLayerConvNet(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)  # 计算输出分数
    print(scores.size())  # 应该输出 [64, 10]
test_ThreeLayerConvNet()  # 调用测试函数

### 模块 API：检查准确率
给定验证集或测试集，我们可以检查神经网络的分类准确率。

这个版本与第二部分中的版本略有不同。你不再需要手动传入参数。

In [ ]:
def check_accuracy_part34(loader, model):
    # 判断是验证集还是测试集
    if loader.dataset.train:
        print('正在验证集上检查准确率')
    else:
        print('正在测试集上检查准确率')
    num_correct = 0  # 正确预测的数量
    num_samples = 0  # 总样本数量
    model.eval()  # 将模型设置为评估模式（关闭 dropout 等训练时特有的层）
    # 禁用梯度计算
    with torch.no_grad():
        for x, y in loader:
            # 将输入数据移动到指定设备（如 GPU）并转换为指定数据类型
            x = x.to(device=device, dtype=dtype)
            # 将标签移动到指定设备并转换为长整数类型
            y = y.to(device=device, dtype=torch.long)
            # 计算模型输出分数
            scores = model(x)
            # 找到分数最高的类别作为预测结果
            _, preds = scores.max(1)
            # 累加正确预测的数量和总样本数量
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
            
        # 计算准确率
        acc = float(num_correct) / num_samples
        print('正确数：%d / 总数：%d（准确率：%.2f%%）' % (num_correct, num_samples, 100 * acc))

### 模块 API：训练循环
我们还使用了一个略有不同的训练循环。我们不再自己更新权重值，而是使用 `torch.optim` 包中的 Optimizer（优化器）对象，它封装了优化算法的概念，并提供了大多数常用于优化神经网络的算法实现。

In [ ]:
def train_part34(model, optimizer, epochs=1):
    """
    使用PyTorch的Module API在CIFAR-10上训练模型。

    输入：
    - model：要训练的PyTorch Module模型
    - optimizer：用于训练模型的Optimizer对象
    - epochs：（可选）训练的轮次（Python整数）

    返回：无返回值，但会在训练过程中打印模型的准确率。
    """
    model = model.to(device=device)  # 将模型参数移动到CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # 将模型设置为训练模式
            x = x.to(device=device, dtype=dtype)  # 移动到设备（如GPU）并转换数据类型
            y = y.to(device=device, dtype=torch.long)  # 移动标签到设备并转换为长整数类型

            scores = model(x)  # 前向传播计算分数
            loss = F.cross_entropy(scores, y)  # 计算交叉熵损失

            # 将优化器要更新的所有变量的梯度清零
            optimizer.zero_grad()

            # 反向传播：计算损失相对于模型每个参数的梯度
            loss.backward()

            # 使用反向传播计算出的梯度实际更新模型参数
            optimizer.step()

            # 定期打印训练信息和验证集准确率
            if t % print_every == 0:
                print('迭代次数 %d，损失 = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

### 模块 API：训练一个两层网络
现在我们可以运行训练循环了。与第二部分不同，我们不再需要显式地分配参数张量。

只需将输入大小、隐藏层大小和类别数（即输出大小）传递给`TwoLayerFC`的构造函数即可。

你还需要定义一个优化器，用于跟踪`TwoLayerFC`中所有可学习的参数。

你不需要调整任何超参数，但训练一个轮次后，模型准确率应该能达到40%以上。

In [ ]:
hidden_layer_size = 4000  # 隐藏层大小
learning_rate = 1e-2  # 学习率

# 实例化两层全连接网络：输入大小为3*32*32（展平后的图像），隐藏层大小为4000，输出类别数为10
model = TwoLayerFC(3 * 32 * 32, hidden_layer_size, 10)

# 定义优化器：使用SGD优化器，跟踪模型中的所有参数，学习率为上面定义的值
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 训练模型
train_part34(model, optimizer)

### 模块 API：训练一个三层卷积网络
现在你应该使用模块 API 在 CIFAR 数据集上训练一个三层卷积网络。这应该和训练两层网络非常相似！你不需要调整任何超参数，但训练一个轮次后，准确率应该能达到 45% 以上。

你应该使用不带动量的随机梯度下降来训练这个模型。

In [ ]:
learning_rate = 3e-3  # 学习率
channel_1 = 32  # 第一层卷积输出通道数
channel_2 = 16  # 第二层卷积输出通道数

model = None  # 三层卷积网络模型实例
optimizer = None  # 优化器实例
################################################################################
# 待办：实例化你的ThreeLayerConvNet模型和对应的优化器                           #
################################################################################

################################################################################
#                                 你的代码结束                                 #
################################################################################

# 训练模型
train_part34(model, optimizer)

# 第四部分. PyTorch 顺序 API

第三部分介绍了 PyTorch 的 Module API，它允许你定义任意的可学习层及其连接方式。

对于像堆叠的前馈层这样的简单模型，你仍然需要经过三个步骤：继承 `nn.Module`、在 `__init__` 中将层分配为类属性、在 `forward()` 中逐个调用每个层。有没有更便捷的方法呢？

幸运的是，PyTorch 提供了一个名为 `nn.Sequential` 的容器模块，它将上述步骤合并为一个。它不像 `nn.Module` 那样灵活，因为你不能指定比前馈堆叠更复杂的拓扑结构，但对于许多使用场景来说已经足够了。

### 顺序 API：两层网络
让我们看看如何使用 `nn.Sequential` 重写我们的两层全连接网络示例，并使用上面定义的训练循环来训练它。

同样，这里你不需要调整任何超参数，但训练一个轮次后，准确率应该能达到 40% 以上。

In [ ]:
# 我们需要将`flatten`函数包装在一个模块中，以便在nn.Sequential中堆叠使用
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)  # 展平输入张量

hidden_layer_size = 4000  # 隐藏层大小
learning_rate = 1e-2  # 学习率

# 使用nn.Sequential构建两层全连接网络
model = nn.Sequential(
    Flatten(),  # 展平操作
    nn.Linear(3 * 32 * 32, hidden_layer_size),  # 第一层全连接层
    nn.ReLU(),  # ReLU激活函数
    nn.Linear(hidden_layer_size, 10),  # 第二层全连接层（输出层）
)

# 可以在optim.SGD中使用Nesterov动量
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

# 训练模型
train_part34(model, optimizer)

### 顺序 API：三层卷积网络
在这里，你需要使用`nn.Sequential`来定义和训练一个三层卷积网络，其架构与我们在第三部分中使用的相同：

1. 带偏置的卷积层，包含32个5x5的滤波器，零填充为2
2. ReLU激活函数
3. 带偏置的卷积层，包含16个3x3的滤波器，零填充为1
4. ReLU激活函数
5. 带偏置的全连接层，用于计算10个类别的分数

你可以使用PyTorch默认的权重初始化方式。

你应该使用带有Nesterov动量（动量值为0.9）的随机梯度下降来优化模型。

同样，你不需要调整任何超参数，但训练一个轮次后，准确率应该能达到55%以上。

In [ ]:
channel_1 = 32  # 第一层卷积输出通道数
channel_2 = 16  # 第二层卷积输出通道数
learning_rate = 1e-2  # 学习率

model = None  # 三层卷积网络模型（使用Sequential API构建）
optimizer = None  # 优化器

################################################################################
# 待办：使用Sequential API重写第三部分中带偏置的两层卷积网络。                   #
# （注：原文"2-layer ConvNet"应为笔误，结合上下文应为三层卷积网络）              #
################################################################################

################################################################################
#                                 你的代码结束                                 #
################################################################################

# 训练模型
train_part34(model, optimizer)

# 第五部分. CIFAR-10 开放式挑战

在本部分中，你可以在 CIFAR-10 数据集上尝试任何你喜欢的卷积网络架构。

现在，你的任务是通过尝试不同的架构、超参数、损失函数和优化器来训练模型，使其在 10 个轮次内，在 CIFAR-10 **验证集**上达到**至少 70%** 的准确率。你可以使用上面的 check_accuracy 函数和 train 函数。你可以使用 `nn.Module` 或 `nn.Sequential` API。

在本笔记本的末尾描述你所做的工作。

以下是每个组件的官方 API 文档。注意：我们在课堂上所说的“空间批归一化（spatial batch norm）”在 PyTorch 中称为“BatchNorm2D”。

* torch.nn 包中的层：http://pytorch.org/docs/stable/nn.html
* 激活函数：http://pytorch.org/docs/stable/nn.html#non-linear-activations
* 损失函数：http://pytorch.org/docs/stable/nn.html#loss-functions
* 优化器：http://pytorch.org/docs/stable/optim.html


### 你可以尝试的方向：
- **滤波器大小**：上面我们使用了 5x5 的滤波器；更小的滤波器会更高效吗？
- **滤波器数量**：上面我们使用了 32 个滤波器。更多或更少的滤波器会表现更好吗？
- **池化与步长卷积**：你会使用最大池化还是步长卷积？
- **批归一化**：尝试在卷积层后添加空间批归一化，在仿射层后添加普通批归一化。你的网络训练速度会更快吗？
- **网络架构**：上面的网络有两层可训练参数。更深的网络会表现更好吗？值得尝试的良好架构包括：
    - [卷积-激活-池化]xN -> [仿射]xM -> [softmax 或 SVM]
    - [卷积-激活-卷积-激活-池化]xN -> [仿射]xM -> [softmax 或 SVM]
    - [批归一化-激活-卷积]xN -> [仿射]xM -> [softmax 或 SVM]
- **全局平均池化**：不是先展平然后使用多个仿射层，而是执行卷积直到图像变得很小（大约 7x7），然后执行平均池化操作得到 1x1 的图像（1, 1, 滤波器数量），再将其重塑为（滤波器数量）维的向量。这在[谷歌的 Inception 网络](https://arxiv.org/abs/1512.00567)中使用（参见表 1 了解其架构）。
- **正则化**：添加 l2 权重正则化，或者可能使用 Dropout。

### 训练技巧
对于你尝试的每个网络架构，你都应该调整学习率和其他超参数。在这过程中，有几个重要的事情需要记住：

- 如果参数设置合理，你应该在几百次迭代内看到性能提升
- 记住超参数调优的“由粗到细”方法：先在较大的超参数范围内测试，只训练几次迭代，找出那些有效的参数组合。
- 一旦你找到了一些似乎有效的参数集，在这些参数周围进行更精细的搜索。你可能需要训练更多的轮次。
- 你应该使用验证集进行超参数搜索，并保留测试集用于在由验证集选出的最佳参数上评估你的架构。

### 更进一步
如果你有兴趣，还有许多其他功能可以实现来尝试提高性能。你**不需要**实现这些，但如果有时间，不妨尝试一下，会很有趣！

- 替代优化器：你可以尝试 Adam、Adagrad、RMSprop 等。
- 替代激活函数，如 leaky ReLU、参数化 ReLU、ELU 或 MaxOut。
- 模型集成
- 数据增强
- 新架构
  - [ResNets](https://arxiv.org/abs/1512.03385)，其中前一层的输入被添加到输出中。
  - [DenseNets](https://arxiv.org/abs/1608.06993)，其中前几层的输入被连接在一起。
  - [这个博客有深入的概述](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

### 祝你训练愉快！

In [ ]:
################################################################################
# 待办：                                                                        #
# 尝试各种架构、优化器和超参数。                                                 #
# 在10个轮次内，在*验证集*上达到至少70%的准确率。                                 #
#                                                                              #
# 注意，你可以使用check_accuracy函数来评估测试集或验证集，通过将loader_test或     #
# loader_val作为第二个参数传递给check_accuracy。在完成架构和超参数调整之前，      #
# 不应触碰测试集，只在最后运行一次测试集以报告最终结果。                          #
################################################################################
model = None  # 模型实例
optimizer = None  # 优化器实例

################################################################################
#                                 你的代码结束                                 #
################################################################################

# 你的模型应该至少达到70%的准确率
train_part34(model, optimizer, epochs=10)  # 训练模型，共10个轮次

## 描述你所做的工作

在下面的单元格中，你应该写下对自己所做工作的说明，包括你实现的任何额外功能，和/或在训练与评估网络过程中绘制的任何图表。

**答案：**



## 测试集——仅运行一次

既然我们已经得到了满意的结果，现在就用我们的最终模型在测试集上进行测试（你应该将最终模型存储在 best_model 中）。思考一下这与你的验证集准确率相比如何。

In [ ]:
best_model = model  # 将训练好的最佳模型赋值给best_model
# 在测试集上检查最佳模型的准确率
check_accuracy_part34(loader_test, best_model)